In [2]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme
from tqdm import tqdm

In [3]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/integration/integration2.log", filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [4]:
from coralme.solver.solver import ME_NLP

def get_nlp(model):
    Sf, Se, lb, ub, b, c, cs, atoms, lambdas = model.construct_lp_problem(lambdify=True)
    me_nlp = ME_NLP(Sf, Se,b, c, lb, ub,  cs, atoms, lambdas)
    return me_nlp

def get_feasibility(me_nlp, basis=None):
    x_new,y_new,z_new,stat_new,hs_new = me_nlp.solvelp(0.001,basis,'quad')
    return (True,hs_new) if stat_new=="optimal" else (False,hs_new)

def knockout(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        me_nlp.Sf[(m_idx,ri)] = 0

def restore(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        me_nlp.Sf[(m_idx,ri)] = 1

def printcoeff(model,met,me_nlp):
    # met = "RNA_592010.4.peg.10"
    rxns = [r.id for r in model.metabolites.get_by_id(met).reactions if "transcription" in r.id]
    m_idx = model.metabolites.index(met)
    r_idxs = [model.reactions.index(r) for r in rxns ]
    for ri in r_idxs:
        # print(tmp.Sf[(m_idx,ri)])
        print(me_nlp.Sf[(m_idx,ri)])

def get_targets(model,condition):
    model_genes = [i.id.split("RNA_")[1] for i in model.all_genes]
    return [g for g in model_genes if g not in NormalizedTCounts.index or NormalizedTCounts.loc[g][condition]<1e-16]

def get_killable(model,me_nlp,basis,kill_genes):
    killable = []
    for idx,k in enumerate(kill_genes):
        k = "RNA_" + k
        knockout(model,k,me_nlp)
        f,new_basis = get_feasibility(me_nlp,basis=basis)
        if f:
            ListHandler.print_and_log("{} feasible if {} knockout ({} out of {})".format(model.id,k,idx+1,len(kill_genes)))
            killable.append(k)
            basis = new_basis
            continue
        ListHandler.print_and_log("{} not feasible if {} knockout ({} out of {})".format(model.id,k,idx+1,len(kill_genes)))
        restore(model,k,me_nlp)
    ListHandler.print_and_log("Done with {}".format(model.id))
    return killable

In [5]:
MeanTCounts = pandas.read_csv("./ibdmdb/metaT_per_diagnosis.csv",index_col=0)
NormalizedTCounts = MeanTCounts.div(MeanTCounts.sum())
NormalizedTCounts.head()

,nonIBD,UC,CD,IBD
#FeatureID,,,,
HMPREF9436_RS14285,0.078333,0.039315,0.037492,0.037492
HMPREF9436_RS14685,0.023592,0.008788,0.013451,0.013451
BILO145876EF_RS04385,0.022452,0.003468,0.005717,0.005717
FAEPRAM212_RS14865,0.017201,0.023327,0.030562,0.030562
BSFG_RS23450,0.010633,0.011724,0.010224,0.010224


In [6]:
# org = "Abiotrophia_defectiva_ATCC_49176"
condition = "nonIBD"
# condition = "IBD"
# condition = "UC"
# condition = "CD"

In [7]:
run_for = {'Kytococcus_sedentarius_DSM_20547',
 'Listeria_monocytogenes_Finland_1988',
 'Providencia_stuartii_ATCC_25827',
 'Ruminococcus_torques_ATCC_27756'}

In [7]:
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    me_nlp = get_nlp(model)
    kill_genes = get_targets(model,condition)
    f,basis = get_feasibility(me_nlp)
    killable = get_killable(model,me_nlp,basis,kill_genes)
    pandas.DataFrame(columns=killable).T.to_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None)

In [ ]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
    
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (4 threads):   0%|                                                                                   | 0/4 [00:00<?, ?it/s]

Loading Ruminococcus_torques_ATCC_27756Loading Listeria_monocytogenes_Finland_1988Loading Kytococcus_sedentarius_DSM_20547Loading Providencia_stuartii_ATCC_25827



Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp2pnmbelw.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp48qybyai.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp6byo9uvp.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpwfg4r_y1.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from f

Building (4 threads):  25%|██████████████████▌                                                       | 1/4 [10:49<32:29, 649.95s/it]

Providencia_stuartii_ATCC_25827-ME feasible if RNA_471874.6.peg.911 knockout (174 out of 1111)
Listeria_monocytogenes_Finland_1988-ME feasible if RNA_393127.9.peg.501 knockout (144 out of 952)
Listeria_monocytogenes_Finland_1988-ME feasible if RNA_393127.9.peg.502 knockout (145 out of 952)
Providencia_stuartii_ATCC_25827-ME feasible if RNA_471874.6.peg.912 knockout (175 out of 1111)
Listeria_monocytogenes_Finland_1988-ME feasible if RNA_393127.9.peg.503 knockout (146 out of 952)
Providencia_stuartii_ATCC_25827-ME feasible if RNA_471874.6.peg.920 knockout (176 out of 1111)
Kytococcus_sedentarius_DSM_20547-ME feasible if RNA_478801.4.peg.646 knockout (161 out of 709)
Listeria_monocytogenes_Finland_1988-ME not feasible if RNA_393127.9.peg.504 knockout (147 out of 952)
Kytococcus_sedentarius_DSM_20547-ME not feasible if RNA_478801.4.peg.647 knockout (162 out of 709)
Providencia_stuartii_ATCC_25827-ME feasible if RNA_471874.6.peg.921 knockout (177 out of 1111)
Listeria_monocytogenes_Finland